# <center>Delinear oportunidades colaboração</center>
### <center>Pasteur(Fr) & Fiocruz(Ce)</center>

In [1]:
## Configurar opções de exibição do pandas para melhor visualizar os dados
from pathlib import Path
from getpass import getpass
from datetime import datetime
from IPython.display import clear_output
import pandas as pd, os, re, sys, time, json, subprocess
pd.set_option('display.max_colwidth', None)
pd.set_option('colheader_justify', 'left')
pd.set_option('display.max_rows', 600)

def find_repo_root(path='.', depth=10):
    ''' 
    Busca o arquivo .git e retorna string com a pasta raiz do repositório
    '''
    # Prevent infinite recursion by limiting depth
    if depth < 0:
        return None
    path = Path(path).absolute()
    if (path / '.git').is_dir():
        return path
    return find_repo_root(path.parent, depth-1)

delay = 10

## Definir a pasta de base do repositório local
base_repo_dir = find_repo_root()

## Sempre construir os caminhos usando os.path.join para compatibilidade WxL
folder_utils = os.path.join(base_repo_dir, 'utils')
folder_domain = os.path.join(base_repo_dir, 'source', 'domain')
folder_data_input = os.path.join(base_repo_dir, '_data', 'in_csv')
folder_data_output = os.path.join(base_repo_dir, '_data', 'out_json')

## Adicionar pastas locais ao sys.path para importar pacotes criados localmente
sys.path.append(folder_utils)
sys.path.append(folder_domain)
from chromedriver_manager import ChromeDriverManager
from json_fle_manager import JSONFileManager as jfm
from scraper_pasteur import PasteurScraper
from scraper_fiocruz_ceara import FiocruzCearaScraper
from lattes_scrapper import LattesScraper, SoupParser, DictToHDF5, attribute_to_be_non_empty
# from environment_setup import EnvironmentSetup
# from lattes_html_parser import HTMLParser

print(f" Caminho base do repositório: {base_repo_dir}")
print(f"Arquivos de entrada de dados: {jfm.list_json_files(folder_data_input)}")

 Caminho base do repositório: c:\Users\marcos.aires\ppgcs
Arquivos de entrada de dados: ['dict_list_complete.json', 'dict_list_discents.json', 'dict_list_docents.json', 'dict_list_temp.json']


In [2]:
# Cria instância da classe ChromeDriverManager e verifica compatibilidade entre versões do Chrome e Chromedriver
manager = ChromeDriverManager()
manager.main()

Versões 124 Chrome e 124 Chromedriver estão compatíveis


# Dados Fiocruz Ceará: 

- Áreas 
- Linhas 
- Pesquisadores

### Extrair linhas de pesquisa e pesquisadores de cada área

In [3]:
base_url = "https://ceara.fiocruz.br/portal/"
scraper = FiocruzCearaScraper(base_url, find_repo_root())
fioce_data = scraper.scrape_areas()
fioce_data

URL pesquisada:
https://ceara.fiocruz.br/portal/index.php/pesquisa/saude-da-familia/pesquisadores/
URL pesquisada:
https://ceara.fiocruz.br/portal/index.php/pesquisa/saude-e-ambiente/pesquisadores/
URL pesquisada:
https://ceara.fiocruz.br/portal/index.php/pesquisa/biotecnologia/pesquisadores/
URL pesquisada:
https://ceara.fiocruz.br/portal/index.php/pesquisa/saude-digital/pesquisadores-em-saude-digital/


[{'area_name': 'Saúde da Família',
  'area_desc': 'As pesquisas da área avaliam a Atenção Primária à Saúde. São estudos que investigam a atuação das equipes da Estratégia Saúde da Família em diferentes cenários e/ou processos de trabalho e/ou cuidado, em distintos ciclos de vida da população. As temáticas de pesquisa incluem o ensino e as práticas dos profissionais da saúde, saúde mental, condições crônicas, saúde bucal e saúde digital.',
  'area_plats': ['As diversas iniciativas que no momento caracterizam a atuação da Área de Saúde da Família na Fiocruz Ceará consolidam um processo de construção participativo, dinâmico e voltado a garantir respostas aos desafios de fortalecimento e qualificação dessa estratégia que, em virtude de sua magnitude, impõe à sociedade e governos, gigantesco esforços ao efetivo cumprimento de seu propósito. Entende-se pela sua magnitude: a cobertura de mais de 159 milhões de pessoas; mais de 1 milhão de postos de trabalho diretos; cobertura de quase todos o

In [4]:
for i in fioce_data:
    print(i.get('area_pesquisadores'))

['Anya Pimentel Gomes Fernandes Vieira Meyer', 'Ivana Cristina de Holanda Cunha Barreto', 'Luiz Odorico Monteiro de Andrade', 'Márcio Flávio Moura de Araújo', 'Maximiliano Loiola Ponte de Souza', 'Roberto Wagner Júnior Freire de Freitas', 'Sharmênia de Araújo Soares Nuto', 'Vanira Matos Pessoa']
['Ana Cláudia de Araújo Teixeira', 'Fernando Ferreira Carneiro', 'José Luís Passos Cordeiro', 'Margareth Borges Coutinho Gallo']
['Anna Carolina Machado Marinho', 'Carla Freire Celedonio Fernandes', 'Clarissa Romero Teixeira', 'Fábio Miyajima', 'Fernando Braga Stehling Dias', 'Gilvan Pessoa Furtado', 'Jaime Ribeiro Filho', 'João Herminio Martins da Silva', 'Marcos Roberto Lourenzoni', 'Raphael Trevizani', 'Regis Bernardo Brandim Gomes', 'Roberto Nicolete']
['Adriana Costa Bacelo', 'Galba Freire Moita', 'Ivana Cristina De Holanda Cunha Barrêto', 'Luiz Odorico Monteiro de Andrade']


In [5]:
for i in fioce_data:
    print(i.get('area_lines'))

['- Doenças Crônicas e não-transmissíveis, medicina regenerativa', '- Pesquisa Clínica e Ensaios Clínicos', '- Saúde e Gênero, Saúde do Idoso', '- Saúde Perinatal, da Criança e do Adolescente', '- Epidemiologia, métodos estatísticos e quantitativos', '- Políticas Públicas, Planejamento e Gestão em Saúde', '- Gestão de Ciência e Tecnologia em Saúde', '- Promoção da Saúde', '- Avaliação e Economia da Saúde', '- Sociologia, Antropologia, Filosofia e Saúde, Cultura e Sociedade', '- Educação em Ciências e Saúde', '- Políticas Públicas para Saúde em Territórios', '- Tecnologia de informação e comunicação']
['- Vigilância Popular da Saúde, Ambiente e Trabalho', '- Saúde, Agroecologia e Convivência com o Semiárido', '- Impactos dos grandes empreendimentos na saúde', '- Atenção Primária em Saúde, Ambiente e Trabalho', '- Saúde Única/Saúde Planetária (One Health/Global Health)', '- Metodologias de pesquisa crítico-dialógicas e participativas', '- Populações do campo, floresta, águas e áreas urba

### Extrair linhas de pesquisa de cada pesquisador

In [6]:
linhas_pesquisadores = scraper.scrape_lines()


Pesquisando linhas da área: Biotecnologia...
https://ceara.fiocruz.br/portal/index.php/pesquisa/biotecnologia/pesquisadores/

Pesquisando linhas da área: Saude Da Familia...
https://ceara.fiocruz.br/portal/index.php/pesquisa/saude-da-familia/linhas-de-pesquisa/

Pesquisando linhas da área: Saude E Ambiente...
https://ceara.fiocruz.br/portal/index.php/pesquisa/saude-e-ambiente/linhas-de-pesquisa/

Pesquisando linhas da área: Saude Digital...
https://ceara.fiocruz.br/portal/index.php/pesquisa/saude-digital/pesquisadores-em-saude-digital/


------------------------------------------------------------------------------------------------------------------------
LINHAS DE CADA PESQUISADOR

Anna Carolina Machado Marinho
– Abordagem proteômica para avaliação e estudo de proteoformas
– Expressão, purificação, caracterização estrutural e ensaios de estabilidade de nanocorpos e construtos relacionados para doenças tropicais negligenciadas
– Avaliação e caracterização de biofármacos possibilitand

In [7]:
linhas_pesquisadores

{'Anna Carolina Machado Marinho': ['– Abordagem proteômica para avaliação e estudo de proteoformas',
  '– Expressão, purificação, caracterização estrutural e ensaios de estabilidade de nanocorpos e construtos relacionados para doenças tropicais negligenciadas',
  '– Avaliação e caracterização de biofármacos possibilitando alinhar o desenvolvimento de metodologias analíticas modernas (espectrometria de massas) com PD&I em produtos biológicos'],
 'Carla Freire Celedonio Fernandes': ['– Pesquisa e Desenvolvimento de biofármacos, anticorpos terapêuticos, com ênfase em nanocorpos',
  '– Pesquisa, Desenvolvimento de novas tecnologias para diagnósticos rápidos, miniaturizados, automação, moleculares, biossensores, nanotecnologias para diagnóstico',
  '– Bioprospecção e bioensaios'],
 'Clarissa Romero Teixeira': ['– Imunopatogênese das Leishmanioses',
  '– Resposta imune contra a saliva de vetores artrópodes',
  '– Prospecção e caracterização de novos tratamentos e candidatos a vacina contra a

# Dados Instituto Pasteur FR: 

- Prioridades
- Plataformas
- Centros
- Times

In [8]:
base_url = "https://research.pasteur.fr"
scraper = PasteurScraper(base_url, find_repo_root())
json_data = scraper.generate_pasteur_report_html()

2024-05-15 09:22:14,172 - INFO - Obtendo os dados do site do Instituto Pasteur, aguarde...
2024-05-15 09:22:15,671 - INFO - 5 seções de dados principais extraídas
2024-05-15 09:22:15,674 - INFO - Obtendo áreas prioritárias de pesquisa, aguarde...
2024-05-15 09:22:26,185 - INFO - 5 áreas prioritárias em pesquisa extraídas
2024-05-15 09:22:26,189 - INFO - Obtendo os centros de referência, aguarde...
2024-05-15 09:22:29,972 - INFO - 5 centros extraídos
2024-05-15 09:22:29,975 - INFO - Obtendo os departamentos, aguarde...
2024-05-15 09:22:54,043 - INFO - 13 departamentos extraídos
2024-05-15 09:22:54,047 - INFO - Obtendo as plataformas, aguarde...
2024-05-15 09:23:53,264 - INFO - 35 plataformas extraídas
2024-05-15 09:23:53,279 - INFO - Relatório concluído!
2024-05-15 09:23:53,282 - INFO - Salvo em: c:\Users\marcos.aires\ppgcs\templates\report_pasteur_fr.html


## Visualizar dados extraídos Instituto Pasteur FR

In [ ]:
json_data.keys()

In [ ]:
json_data.get('platforms_data')[0]

In [ ]:
for i in json_data.get('platforms_data'):
    platform_title = i.get('title')
    print('-'*50)
    print(platform_title.upper())
    try:
        project_list = i.get('aditional_info').get('projects')
        for j in project_list:
            title = j.get('title')
            head = j.get('head_name')
            status = j.get('status')
            description = j.get('description')
            print(f"  [{status}] {title} ({head})")
            print(f"            {description}")
    except:
        try:
            project_list = i.get('aditional_info').get('transversal_projects')
            for j in project_list:
                print(f"[Transversal Project] {j.get('title')}")
        except:
            print(f"            Projetos não encontrados para esta plataforma")

In [ ]:
for i in json_data.get('departments_data'):
    for j in i.get('aditional_info').get('teams'):
        print(j)
        print(f"  {j.get('head_name'):>35}: <b>{j.get('team_name')}</b>")
    print('-'*150)

### Recuperar dados quantitativos gerais

In [ ]:
scraper = PasteurScraper("https://research.pasteur.fr", base_repo_dir)
main_data = scraper.scrape_main_page_quantitative_data()
print(f"{len(main_data)} seções de dados principais extraídas")
for i,j in main_data.items():
    print(f"{i:>25}: {j}")

### Recuperar dados das Áreas Prioritárias de Pesquisa

In [ ]:
from pprint import pprint
priority_research = scraper.scrape_priority_scientific_areas()
filepath = os.path.join(folder_data_output,'pasteur_fr_priority_research.json')
scraper.save_to_json(priority_research, filepath)

print(f"{len(priority_research)} prioridades principais em pesquisa extraídas")
for i in priority_research:
    print(f"  {i.get('area_name'):>50}: {i.get('team_count')}")
print()

In [ ]:
priority_research

## Recuperar dados dos Departamentos do Pasteur

In [ ]:
departments_data = scraper.scrape_department_data()
filepath = os.path.join(folder_data_output,'pasteur_fr_departments_data.json')
scraper.save_to_json(departments_data, filepath)

print(f"{len(departments_data)} departamentos extraídos")
for i in departments_data:
    print(f"  {i.get('head_name'):>25}: {i.get('department_name')}")

In [ ]:
departments_data

## Recuperar dados dos Centros do Pasteur

In [ ]:
heads_centers_data = scraper.scrape_centers()
centers_data = scraper.scrape_centers_data()
filepath = os.path.join(folder_data_output,'pasteur_fr_centers_data.json')
scraper.save_to_json(centers_data, filepath)

print(f"{len(centers_data)} centros extraídos")
for i in heads_centers_data:
    print(f"  {i.get('head_name'):>35}: {i.get('title')}")
for i in centers_data:
    print(f"\n{i.get('center_title')} ({len(i.get('teams')):02} times associados)")
    for team in i.get('teams'):
        print(f"  {team.get('head_name'):>35}: {team.get('title')}")

## Recuperar dados das Plataformas do Pasteur

In [ ]:
platforms_data = scraper.scrape_platforms()
filepath = os.path.join(folder_data_output,'pasteur_fr_platforms_data.json')
scraper.save_to_json(platforms_data, filepath)

print(f"{len(platforms_data)} plataformas extraídas")
for i in platforms_data:
    print(f"  {i.get('head_name'):>25}: {i.get('title')}")

## Recuperar dados dos Times do Pasteur

In [ ]:
teams_data = scraper.scrape_teams_data()
print(f"{len(teams_data)} times extraídos")
for i in teams_data:
    print(f"  {i.get('head_name'):>25}: {i.get('title')}")

In [ ]:
filepath = os.path.join(folder_data_output,'pasteur_fr_teams_data.json')
scraper.save_to_json(teams_data, filepath)

In [ ]:
centers_data[1]

In [ ]:
departments_data[-1]

In [ ]:
platforms_data[-1]

## Outros testes

In [ ]:
# import requests
# from bs4 import BeautifulSoup
# import json

# # Requisição HTTP para obter o conteúdo da página
# url = "https://ceara.fiocruz.br/portal/index.php/pesquisa/biotecnologia/pesquisadores/"
# response = requests.get(url)

# # Verificação do status da requisição
# if response.status_code == 200:
#     # Conteúdo HTML da página
#     html_content = response.content

#     # Criação do objeto BeautifulSoup
#     soup = BeautifulSoup(html_content, 'lxml')
#     dados_div = {}

#     # Localização das divs com a classe especificada
#     divs = soup.find_all('div', class_="col-sm-12")
#     # divs = soup.find_all('div', style="background-color:#d0d3aa")
#     # divs = soup.find_all('div', style="is-layout-flex wp-container-7 wp-block-columns has-background has-medium-font-size")

#     # Lista para armazenar os dados em JSON
#     dados_json = []

#     # Iteração pelas divs
#     for div in divs:
#         div_pesquisador = div.find('div', class_='is-layout-flex wp-container-7 wp-block-columns has-background has-medium-font-size')
#         # if div_pesquisador:
#         #     if div_pesquisador.get_text():
#         #         print(div_pesquisador.get_text().strip())
#         # Dicionário para armazenar os dados de cada div
#         dados_div = {}

#         # Nome
#         nome_tag = div.find('p', class_='has-black-color has-text-color has-medium-font-size')
#         if nome_tag:
#             nome = nome_tag.text.strip()
#             dados_div['nome'] = nome

#         # Identificação em publicações
#         identificacao_tag = div.find('p', class_='has-small-font-size')
#         if identificacao_tag:
#             texto_identificacao = identificacao_tag.text.strip()
#             if 'Identificação em publicações:' in texto_identificacao:
#                 identificacao = texto_identificacao.split(':')[1].strip()
#                 dados_div['identificacao'] = identificacao

#         # Contato
#         contato_tag = div.find('p', class_='has-small-font-size')
#         if contato_tag:
#             texto_contato = contato_tag.find_next_sibling('p').text.strip()
#             if 'Contato:' in texto_contato:
#                 contato = texto_contato.split(':')[1]
#                 if contato:
#                     dados_div['contato'] = contato

#         # Currículo Lattes
#         curriculo_lattes_tag = div.find('p', class_='has-small-font-size')
#         if curriculo_lattes_tag:
#             try:
#                 link_lattes = curriculo_lattes_tag.find_next_sibling('p').find('a')['href']
#                 if link_lattes:
#                     dados_div['curriculo_lattes'] = link_lattes
#             except:
#                 pass
            
#         # Formação e experiência
#         formacao_experiencia_tag = div.find_all_next('p', class_='has-small-font-size')
#         if formacao_experiencia_tag:
#             texto_formacao_experiencia = '\n'.join([p.text.strip() for p in formacao_experiencia_tag])
#             dados_div['formacao_experiencia'] = texto_formacao_experiencia.split('\n')[1:]

#         # Linhas de pesquisa
#         linha_pesquisa_tag = div.find('h4')
#         if linha_pesquisa_tag:
#             linha_pesquisa_text = linha_pesquisa_tag.find_next_sibling('p').text.strip()
#             if linha_pesquisa_text:
#                 dados_div['linha_pesquisa'] = linha_pesquisa_text

#         # Áreas de pesquisa
#         areas_pesquisa_tag = div.find('h6')
#         if areas_pesquisa_tag:
#             texto_areas_pesquisa = areas_pesquisa_tag.find_next_sibling('p').text.strip()
#             if ' – ' in texto_areas_pesquisa:
#                 areas_pesquisa = texto_areas_pesquisa.split(' – ')
#                 dados_div['areas_pesquisa'] = areas_pesquisa

#         # Adiciona os dados da div à lista
#         dados_json.append(dados_div)

#     # Conversão da lista para JSON
#     json_data = json.dumps(dados_json, ensure_ascii=False, indent=4)

#     # Impressão do JSON formatado
#     print(json_data)

# else:
#     print(f"Erro ao requisitar a página: {response.status_code}")


In [ ]:
# dados_div